## Notebook to test SfM pipeline

1. The repo is already a git submodule so just git submodule update --init --recursive. 
2. Follow the other steps to get the dependencies here: https://www.opensfm.org/docs/building.html, use this file for installing dependencies https://github.com/paulinus/opensfm-docker-base/blob/master/Dockerfile.python3.
    1. Heads up you'll want to do the pip install from within the lcnn conda environment (or another one).
3. To test your install run:
    1. bin/opensfm_run_all data/berlin from the open_sfm directory
    2. python -m http.server and hit the link here: http://localhost:8000/viewer/reconstruction.html#file=/data/berlin/reconstruction.meshed.json
        1. Make sure you run python -m http.server from the OpenSfM directory

In [ ]:
import sys, cv2, os
import numpy as np
import matplotlib.pyplot as plt
# from yaml import load, dump
# try:
#     from yaml import CLoader as Loader, CDumper as Dumper
# except ImportError:
#     from yaml import Loader, Dumper
import yaml
sys.path.append('../OpenSfM')
from opensfm import features, config
from sfm import utils as sfm_util
import importlib
import utils

### To test your own images 

Create a project directory, within which make sure there is a folder named 'images/'

1. run 'python video_converter.py {video} --n {save_every_n_images} --outputdir {path to images/ directory}'
2. copy the config.yaml from OpenSfM/data/berlin to your {filename} directory

### Load Configuration

The first step is to load the config which will contain info on what features to use, how many etc.

It's probably best to just do these from the command line, rather than using Jupyter integration (my jupyter setup doesn't run from within my LCNN environment, I instead include an LCNN kernel for jupyter to run python with).

In [ ]:
file_path = utils.data("door_closed/") #directory containing images and config file
conf = config.load_config(os.path.join(file_path, "config.yaml"))

Extracts metadata from images (not necessary and doesn't work with current video extraction process).
```
$ ../OpenSfM/bin/opensfm extract_metadata $SFM_DATA_DIR
```


Extracts features from every image
```
$ ../OpenSfM/bin/opensfm detect_features $SFM_DATA_DIR
```

In [ ]:
importlib.reload(sfm_util)
"""An Example of How to Load Pre-Detected features from the npz generated from:
                                    ../OpenSfM/bin/opensfm detect_features ../data/lounge"""
for i, filename in enumerate(os.listdir(os.path.join(file_path, "features"))):
    if i % 2 == 0: # display every 2 images
        sfm_util.display_features(file_path, filename, conf, 50)


In [ ]:
"""An example of how to add custom feature points to the npz file containing orb features"""
importlib.reload(sfm_util)

test_insert = False

if test_insert:

    feats_path = file_path+"features/img_0.png.features.npz"
    fts = features.load_features(feats_path, conf)
    print("Before Addition {}".format(fts[0].shape))

    # Generate 200 random points with descriptors and colors
    new_ft_pts = np.random.randint(0, 1080, (200, 4)) # 200 new random points
    new_ft_desc = np.random.randint(0, 255, (200, 32)) 
    new_ft_col = np.random.randint(0, 255, (200, 3)) 

    #Insert the features
    # sfm_util.insert_features((new_ft_pts, new_ft_desc, new_ft_col), feats_path, conf)

    # Check they were added
    fts = features.load_features(file_path+"features/img_0.png.features.npz", conf)
    print("After Addition: {}".format(fts[0].shape))
    assert(np.allclose(fts[0][-200:], new_ft_pts))

Matches features, creates tracks, reconstructs and produces the final mesh file
```
$ ../OpenSfM/bin/opensfm match_features  $SFM_DATA_DIR
$ ../OpenSfM/bin/opensfm create_tracks $SFM_DATA_DIR
$ ../OpenSfM/bin/opensfm reconstruct $SFM_DATA_DIR
$ ../OpenSfM/bin/opensfm mesh $SFM_DATA_DIR
```

### Viewing the mesh. 
#### Note: change ../data/lounge to the directory you are working from

python -m http.server (from main git folder) and hit the link here: http://localhost:8000/OpenSfM/viewer/reconstruction.html#file=/data/lounge/reconstruction.meshed.json

Also, coming next is the extraction of 3D info for the points it's all stored in reconstruction.json.
One problem we may run into is the junction descriptors not being matched between frames?? Then we will never get the 3D info. Also not entirely sure how we will know which points correspond to our junction info. Kidding this is below.

#### Is the point ID in the matched point thing is the index of the points in the initial list of image features? No I think it is the track_id that corresponds to that point. 

Looks like I could check which points correspond to 3D features in each image by checking tracks.csv.

The values of the columns are indicated below (where x, y are in normalized image coordinates). ? could be angle or size of feature.

image, track_id, feature_id, x, y, z, r, g, b

In [ ]:
importlib.reload(sfm_util)
poses = sfm_util.get_camera_poses(file_path) # uses their pose class to get the homogeneous transform
for img in poses.keys():
    print("{} camera pose\n{}".format(img, poses[img])) # (R|t) format

In [ ]:
feats = sfm_util.load_3d_features(file_path)
print(feats.keys())
print(feats['img_0.png'].keys())

%%bash 
../OpenSfM/bin/opensfm undistort $SFM_DATA_DIR

%%bash 
../OpenSfM/bin/opensfm compute_depthmaps $SFM_DATA_DIR

### Viewing PLY Files
sudo apt-get install meshlab -y

done

```bash 
$ ../OpenSfM/bin/opensfm export_ply {proj_dir}
```